# Semana 1:
    Implementar  el  diseño  de  la  aplicación  necesaria  para  organizar  los 
    clasificadores y crear las particiones para los conjuntos de entrenamiento y prueba de un 
    clasificador.  Se  deben  implementar  los  tres  métodos  de  particionado  explicados, 
    validación cruzada, validación simple y validación por boostrap, pero el diseño debería 
    permitir añadir cualquier otro tipo de particionado de forma flexible. Un posible diseño 
    de la aplicación se comenta en el apartado 4. Probar los tres tipos de particionado con el 
    conjunto de datos ballons y con el conjunto tic-tac-toe que ya empleamos en la práctica de iniciación.

In [13]:
# -*- coding: utf-8 -*-
from abc import ABCMeta,abstractmethod
import random
import numpy as np
class Particion():

	# Esta clase mantiene la lista de índices de Train y Test para cada partición del conjunto de particiones  
	def __init__(self):
		self.indicesTrain=[]
		self.indicesTest=[]

	def __init__(self,indicesTrain,indicesTest):
		self.indicesTrain = indicesTrain
		self.indicesTest = indicesTest
		
#####################################################################################################

class EstrategiaParticionado(object):
	
	# Clase abstracta
	__metaclass__ = ABCMeta
	
	# Atributos: deben rellenarse adecuadamente para cada estrategia concreta: nombreEstrategia, numeroParticiones, listaParticiones. Se pasan en el constructor 
	def __init__(self,nombreEstrategia, numeroParticiones,seed=None):
			self.nombreEstrategia = nombreEstrategia
			self.numeroParticiones = numeroParticiones
			random.seed(seed)
	@abstractmethod
	# TODO: esta funcion deben ser implementadas en cada estrategia concreta  
	def creaParticiones(self,datos,seed=None):
		pass
	

#####################################################################################################

class ValidacionSimple(EstrategiaParticionado):

	# Constructor que llama al de la superclase.
	def __init__(self,numeroParticiones, porcentaje,seed=None):
		super(ValidacionSimple,self).__init__("Validacion simple",numeroParticiones)
		self.porcentaje = porcentaje
	
	# Crea particiones segun el metodo tradicional de division de los datos segun el porcentaje deseado.
	# Devuelve una lista de particiones (clase Particion)
	# TODO: implementar
	def creaParticiones(self,datos,seed=None):
		random.seed(seed)
		particiones = []
		ntot = len(datos)
		n = int(ntot*self.porcentaje)
		for i in range(self.numeroParticiones):
			x = list(range(ntot))
			random.shuffle(x)
			p = Particion(x[:n],x[(n+1):])
			particiones.append(p)
		return particiones

			
			
#####################################################################################################      
class ValidacionCruzada(EstrategiaParticionado):
	

	def __init__(self,numFolds,datos,seed=None):

		super(ValidacionCruzada,self).__init__("Validacion cruzada",numFolds,datos)


	# Crea particiones segun el metodo de validacion cruzada.
	# El conjunto de entrenamiento se crea con las nfolds-1 particiones y el de test con la particion restante
	# Esta funcion devuelve una lista de particiones (clase Particion)
	# TODO: implementar
	def creaParticiones(self,datos,seed=None):   
		random.seed(seed)
		particiones = []
		ntot = len(datos)
		x = list(range(ntot))
		n = int(np.floor(ntot/self.numeroParticiones))
		r = ntot%self.numeroParticiones

		for i in range((self.numeroParticiones)):

			if r !=0:
				p = Particion(x[n*i:n*(i+1)+1], x[:n*i]+x[(n*(i+1))+1:])
				particiones.append(p)
			else:
				p = Particion(x[n*i:n*(i+1)], x[:n*i]+x[(n*(i+1)):])
				particiones.append(p)
			r = r-1
		return particiones
		pass
		

#####################################################################################################      
class ValidacionBootstrap(EstrategiaParticionado):
	

	def __init__(self,numFolds,datos,seed=None):
		super().__init__("Validacion simple",numFolds)
	# Crea particiones segun el metodo de validacion por bootstrap.
	# Esta funcion devuelve una lista de particiones (clase Particion)
	# TODO: implementar
	def creaParticiones(self,datos,seed=None):   
		random.seed(seed)
		l = len(datos)
		particiones = []
		for i in range(self.numeroParticiones):
			p1 = list(set(np.random.randint(l,size=l)))
			ints = [i for i in range(l) if i not in p1]
			particiones.append(Particion(p1,ints))
		return particiones
		


In [7]:
import numpy as np
class Datos (object):
    TiposDeAtributos=('Continuo','Nominal')
    # TODO: procesar el fichero para asignar correctamente las variables 
    # tipoAtributos, nombreAtributos,nominalAtributos, datos y diccionarios
    def __init__(self,nombreFichero):
        self.nombreFichero = nombreFichero
        f = open(nombreFichero,'r')
        self.fichero = f.readlines()
        NAtributos = int(self.fichero[0])
        #quitamos el salto de linea y el retorno de carro si es necesario
        aux = self.fichero[1][:-1].rstrip("\r")
        self.nombreAtributos = aux.split(',')
        cols = len(self.nombreAtributos)
        aux = self.fichero[2][:-1].rstrip("\r")
        self.tipoAtributos = aux.split(',')
        
        #Si no es cotinuo o nominal algun tipo marcamos como error
        if any(x != 'Continuo' and x != 'Nominal' for x in self.tipoAtributos):
                raise Exception('ValueError')

        self.nominalAtributos = [x == 'Nominal' for x in self.tipoAtributos]
        
        matriz = list()
        self.diccionarios = []
        aux = [list() for i in range(len(self.nombreAtributos))]
        for x in self.fichero[3:]:
                y = x.rstrip("\r\n").split(',')
                matriz.append(y)

                for i in range(cols):
                        if (y[i] not in aux[i] and self.nominalAtributos[i]):
                                aux[i].append(y[i])
        for x in aux:
                self.diccionarios.append(dict(zip(sorted(x,key=str.lower),range(len(x)))))

        

        self.datos = np.ones((NAtributos,len(self.nombreAtributos)))
        for i in range(NAtributos):
                for j in range(cols):
                        if self.nominalAtributos[j]:
                                self.datos[i,j] = self.diccionarios[j].get(matriz[i][j])
                        else:
                                self.datos[i,j] = matriz[i][j]
        f.close()
            
    def extraeDatos(self, idx):
        return self.datos[idx,:]

In [15]:
datasets = ["balloons.data","tic-tac-toe.data","german.data"]
for z in range(3):
    print(datasets[z])
    dataset=Datos(datasets[z])
    #Anadimos las estrategias para comparar
    
    x1 = ValidacionSimple(1,0.3,dataset).creaParticiones(dataset.datos)
    for x in x1:
        print("SIMPLE:\n","Train\n",x.indicesTrain,"\nTest\n",x.indicesTest)
    x1 = ValidacionCruzada(5,dataset).creaParticiones(dataset.datos)
    for x in x1:
        print("CRUZADA:\n""Train\n",x.indicesTrain,"\nTest\n",x.indicesTest)
    x1 = ValidacionBootstrap(1,dataset).creaParticiones(dataset.datos)
    for x in x1:
        print("BOOTSTRAP:\n""Train\n",x.indicesTrain,"\nTest\n",x.indicesTest)

balloons.data
SIMPLE:
 Train
 [10, 15, 11, 16, 2, 5] 
Test
 [0, 3, 13, 1, 17, 4, 19, 12, 8, 9, 7, 18, 14]
CRUZADA:
Train
 [0, 1, 2, 3] 
Test
 [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CRUZADA:
Train
 [4, 5, 6, 7, 8] 
Test
 [0, 1, 2, 3, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
CRUZADA:
Train
 [8, 9, 10, 11, 12] 
Test
 [0, 1, 2, 3, 4, 5, 6, 7, 13, 14, 15, 16, 17, 18, 19]
CRUZADA:
Train
 [12, 13, 14, 15, 16] 
Test
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 17, 18, 19]
CRUZADA:
Train
 [16, 17, 18, 19] 
Test
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
BOOTSTRAP:
Train
 [0, 1, 2, 3, 4, 5, 6, 9, 10, 12, 13, 15] 
Test
 [7, 8, 11, 14, 16, 17, 18, 19]
tic-tac-toe.data
SIMPLE:
 Train
 [9, 270, 438, 770, 787, 522, 375, 719, 559, 362, 229, 563, 366, 238, 860, 205, 762, 755, 405, 713, 845, 339, 77, 930, 174, 790, 698, 227, 837, 231, 108, 902, 723, 272, 26, 486, 950, 124, 383, 115, 712, 390, 311, 478, 105, 737, 96, 64, 609, 223, 244, 653, 222, 919, 892, 624, 618, 466, 334,

# Semana 2:
      Implementar el clasificador de Naive-Bayes y clasificar los patrones de los siguientes  conjuntos  de  datos  para
      obtener  los  porcentajes  de  error  y  acierto  en clasificación en cada caso, tanto con validación cruzada como
      con validación simple y con validación por bootstrap. 

In [17]:
from abc import ABCMeta,abstractmethod
import numpy as np
from  scipy.stats import norm 
class Clasificador(object):
  
  # Clase abstracta
  __metaclass__ = ABCMeta
  
  # Metodos abstractos que se implementan en casa clasificador concreto
  @abstractmethod
  # TODO: esta funcion deben ser implementadas en cada clasificador concreto
  # datosTrain: matriz numpy con los datos de entrenamiento
  # atributosDiscretos: array bool con la indicatriz de los atributos nominales
  # diccionario: array de diccionarios de la estructura Datos utilizados para la codificacion
  # de variables discretas
  def entrenamiento(self,datosTrain,atributosDiscretos,diccionario):
    pass
  
  
  @abstractmethod
  # TODO: esta funcion deben ser implementadas en cada clasificador concreto
  # devuelve un numpy array con las predicciones
  def clasifica(self,datosTest,atributosDiscretos,diccionario):
    pass
  
  
  # Obtiene el numero de aciertos y errores para calcular la tasa de fallo
  # TODO: implementar
  @staticmethod
  def error(datos,pred):
    # Aqui se compara la prediccion (pred) con las clases reales y se calcula el error    
  	return sum(pred != datos[:,-1])/len(pred)
    
    
  # Realiza una clasificacion utilizando una estrategia de particionado determinada
  # TODO: implementar esta funcion
  def validacion(self,particionado,dataset,clasificador,seed=None):
       
    # Creamos las particiones siguiendo la estrategia llamando a particionado.creaParticiones
    # - Para validacion cruzada: en el bucle hasta nv entrenamos el clasificador con la particion de train i
    # y obtenemos el error en la particion de test i
    # - Para validacion simple (hold-out): entrenamos el clasificador con la particion de train
    # y obtenemos el error en la particion test
    pass
       
  
##############################################################################

class ClasificadorNaiveBayes(Clasificador):

 
  def __init__(self):
    self.tablas = []

  # TODO: implementar
  def entrenamiento(self,datostrain,atributosDiscretos,diccionario,laplace = False):
	
    [f,c] = datostrain.shape
    hipotesis = diccionario[-1]
    l = len(hipotesis)
    continuos = [[] for _ in range(c-1)]
    for i in range(c):
      if(atributosDiscretos[i]):
        self.tablas.append(np.zeros((len(diccionario[i]),l)))
      else:
        self.tablas.append(np.zeros((2,l)))
    #tablas = [np.zeros((len(hipotesis),len(x))) for x in diccionario if len(x) != 0]
    #Contamos las apariencias de cada dato
    #En las columnas las hipotesis
    #Filas los datos discretos o media y varianza si son continuos
    for i in range(f):
      for j in range(c):
        if atributosDiscretos[j]:
          self.tablas[j][int(datostrain[i,j]),int(datostrain[i,c-1])] +=1
        else:
          continuos[j].append(datostrain[i,j])
    #Miramos si hay algun 0 en alguna tabla, si lo hay añadimos 1 a todo

    if laplace :
      for i in range(c):
        if 0 in self.tablas[i] and atributosDiscretos[i]:
          #print( self.tablas[i])
          self.tablas[i] =  self.tablas[i]+1
    i=0
    #Ponemos las medias y varianzas.
    for h in range(l):
      for j in range(c-1):
        if not atributosDiscretos[j]:
          self.tablas[j][0,h] = np.mean(continuos[j])
          self.tablas[j][1,h] = np.std(continuos[j])

  # TODO: implementar
  def clasifica(self,datostest,atributosDiscretos,diccionario):

    [f,c] = datostest.shape
    hipotesis = diccionario[-1]
    l = len(hipotesis)
    clasificacion = []
    for i in range(f):
      p = np.ones(l)#probabilidades de cada hipotesis
      for h in range(l):
        if(sum(self.tablas[0][:,h]) == 0):
          p[h] = 0
          continue;

        for j in range(c-1):
          if(atributosDiscretos[j]):
            p[h] = p[h] * self.tablas[j][int(datostest[i,j]),h]/sum(self.tablas[j][:,h])#Hacemos los a posteriori
          else:
            p[h] = p[h] * norm.pdf(datostest[i,j], loc = self.tablas[j][0,h],scale = self.tablas[j][1,h])

        p[h] =  p[h] * self.tablas[-1][h,h]/sum(sum(self.tablas[-1]))
    
      clasificacion.append(np.where(p == np.max(p))[0][0])
    return clasificacion

    def validacion(self,particionado,dataset,clasificador,seed=None):
      particiones = particionado.creaParticiones(dataset.datos) 
      errores = []
      for x in particiones:
        datosTrain = dataset.extraeDatos(x.indicesTrain)
        datosTest = dataset.extraeDatos(x.indicesTest)
        self.entrenamiento(datosTrain,dataset.nominalAtributos,dataset.diccionarios)  
        valores = self.clasifica(datosTest,dataset.nominalAtributos,dataset.diccionarios)
        errores.apend(error(datosTest,valores))
      return np.mean(errores)      
    

Hemos decidido incluir el codigo de prueba para esta semana junto con el codigo de prueba para la semana 3 ya que asi se puede contrastar con clasificadores que funcionan

# Semana 3:
    Comparar los resultados obtenidos con los que proporciona la librería de aprendizaje automático scikit-learn

In [19]:
from Datos import Datos
from EstrategiaParticionado import *
from Clasificador import *
import numpy as np
from sklearn import preprocessing 
from sklearn.naive_bayes import GaussianNB,MultinomialNB
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn import datasets, linear_model
import warnings
warnings.filterwarnings("ignore")

#Programa para probar el funcionamiento del codigo completo
datasets = ["balloons.data","tic-tac-toe.data","german.data"]
particiones = [20,10,5]
estrategia = []
for z in range(3):
	print(datasets[z])
	dataset=Datos(datasets[z])
	#Anadimos las estrategias para comparar
	
	estrategia.append(ValidacionSimple(particiones[z],0.3,dataset))
	estrategia.append(ValidacionCruzada(particiones[z],dataset))
	estrategia.append(ValidacionBootstrap(particiones[z],dataset))
	#Listas de errores para poder calcular estadisticos
	erroresNV = [[],[],[]]
	erroresL = [[],[],[]]
	erroresSkG = [[],[],[]]
	erroresSkN = [[],[],[]]
	erroresSkGT = [[],[],[]]
	erroresSkNT = [[],[],[]]
	estrategias = ["ValidacionSimple","ValidacionCruzada","ValidacionBootstrap"]
	print("Iteracion \tError Laplace \t Error sin Laplace \t ErrorSklearn(Validacion Simple Validacion Cruzada) (Gausian Multinomial)\n")
	#Procesamiento de los datos para comparar con paquete sklearn
	encAtributos = preprocessing.OneHotEncoder(
		categorical_features = dataset.nominalAtributos[:-1],
		sparse = False)
	X = encAtributos.fit_transform(dataset.datos[:,:-1])
	Y = dataset.datos[:,-1]
	Nominal = [z for z in range(len(dataset.tipoAtributos[:-1])) if dataset.nominalAtributos[z]]
	Continue = [z for z in range(len(dataset.nominalAtributos[:-1])) if not dataset.nominalAtributos[z]]

	for i in range(3):
		print(estrategias[i])

		particion = estrategia[int(i+z*3)].creaParticiones(dataset.datos)	
		j = 0
		for x in particion:
			j +=1
			cnv = ClasificadorNaiveBayes()
			datosTrain = dataset.extraeDatos(x.indicesTrain)
			datosTest = dataset.extraeDatos(x.indicesTest)

			cnv.entrenamiento(datosTrain,dataset.nominalAtributos,dataset.diccionarios)
			NL = cnv.clasifica(datosTest,dataset.nominalAtributos,dataset.diccionarios)
			
			cnvL = ClasificadorNaiveBayes()
			cnvL.entrenamiento(datosTrain,dataset.nominalAtributos,dataset.diccionarios, True)

			L = cnvL.clasifica(datosTest, dataset.nominalAtributos,dataset.diccionarios)
			errorNV = Clasificador.error(datosTest,NL)
			errorL = Clasificador.error(datosTest,L)

			erroresNV[i].append(errorNV)
			erroresL[i].append(errorL)
			#Comprobamos con sklearn
			X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=None)
			#print(X_train)

			gnb = GaussianNB()
			y_pred = gnb.fit(X_train,y_train).predict(X_test)
			errorSkG = (y_test != y_pred).sum()/len(y_pred)
			erroresSkG[i].append(errorSkG)
			gnb = MultinomialNB()
			y_pred = gnb.fit(X_train,y_train).predict(X_test)
			errorSkN = (y_test != y_pred).sum()/len(y_pred)
			erroresSkN[i].append(errorSkN)

			#Vemos las variaciones al tener en cuenta por separado los atributos continuos y los discretos
			gnb = GaussianNB()
			errorNom = 1 - np.mean(cross_val_score(gnb,X, Y,cv=10))
			erroresSkGT[i].append(errorNom)	
			mnb = MultinomialNB()
			errorCon = 1- np.mean(cross_val_score(mnb,X, Y,cv=10))
			erroresSkNT[i].append(errorCon)

			print("{j}\t{l}\t{nl}\t{e}\t{e1}\t{e2}\t{e3}".format(j = j,l = errorL,nl = errorNV,e = errorSkG,e1 = errorSkN,
																														e2 = errorNom, e3 = errorCon),end="\r")

		print("----------------------------------------------------------------------------------------------------------------------------------------------------------",end="\n")
		print("\tSegundo cuantil del error:\n")
		#print(erroresL[i])
		mediaL = np.median(erroresL[i])
		mediaNv = np.median(erroresNV[i])
		mediaSk = np.median(erroresSkG[i])
		mediaSkN = np.median(erroresSkN[i])
		mediaSkT = np.median(erroresSkGT[i])
		mediaSkNT = np.median(erroresSkNT[i])
		print("\t{l}\t{nl}\t{e}\t{e1}\t{e2}\t{e3}\n".format(l = mediaL, nl = mediaNv,e = mediaSk,e1=mediaSkN,e2 = mediaSkT,e3 = mediaSkNT))

balloons.data
Iteracion 	Error Laplace 	 Error sin Laplace 	 ErrorSklearn(Validacion Simple Validacion Cruzada) (Gausian Multinomial)

ValidacionSimple
----------------------------------------------------------------------------------------------------------------------------------------------------------
	Segundo cuantil del error:

	0.23076923076923078	0.3076923076923077	0.0	0.0	0.0	0.0

ValidacionCruzada
----------------------------------------------------------------------------------------------------------------------------------------------------------
	Segundo cuantil del error:

	0.3888888888888889	0.3333333333333333	0.0	0.0	0.0	0.0

ValidacionBootstrap
----------------------------------------------------------------------------------------------------------------------------------------------------------
	Segundo cuantil del error:

	0.0	0.0	0.0	0.0	0.0	0.0

tic-tac-toe.data
Iteracion 	Error Laplace 	 Error sin Laplace 	 ErrorSklearn(Validacion Simple Validacion Cruzada) (Gau